In [12]:
import sys
sys.path.append('../tensorflow/')
from mytensorflow import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

net = Network()
net.add(FCLayer(2, 3))
net.add(ActivationLayer(TangH.callFunction, TangH.callDerivative))
net.add(FCLayer(3, 8))
net.add(ActivationLayer(Relu.callFunction, Relu.callDerivative))
net.add(FCLayer(8, 1))
net.add(ActivationLayer(Sigmoid.callFunction, Sigmoid.callDerivative))


net.compile(MeanSquaredError.fun, MeanSquaredError.derivative)
net.fit(x_train, y_train, epochs=1000, learning_rate=0.1)


out = net.predict(x_train)
print(out)

[array([0.01865309]), array([0.96920372]), array([0.97021151]), array([0.02075516])]


In [3]:
x_train.T.shape

(2, 1, 4)

In [4]:

data_classification = pd.read_csv("../../data/csgo_task_m.csv")
data_classification.drop(["Unnamed: 0"], axis=1, inplace=True)
data_classification

,time_left,ct_score,t_score,bomb_planted,ct_health,t_health,ct_money,t_money,ct_players_alive,t_players_alive,map_de_cache,map_de_dust2,map_de_inferno,map_de_mirage,map_de_nuke,map_de_overpass,map_de_train,map_de_vertigo
0,175.00,0.0,0.0,1,500.0,500.0,4000.0,4000.0,5.0,5.0,0,1,0,0,0,0,0,0
1,156.03,0.0,0.0,1,500.0,500.0,600.0,650.0,5.0,5.0,0,1,0,0,0,0,0,0
2,96.03,0.0,0.0,1,391.0,400.0,750.0,500.0,4.0,4.0,0,1,0,0,0,0,0,0
3,76.03,0.0,0.0,1,391.0,400.0,750.0,500.0,4.0,4.0,0,1,0,0,0,0,0,0
4,174.97,1.0,0.0,1,500.0,500.0,18350.0,10750.0,5.0,5.0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112466,15.41,11.0,14.0,0,200.0,242.0,100.0,5950.0,2.0,4.0,0,0,0,0,0,0,1,0
112467,174.93,11.0,15.0,1,500.0,500.0,11500.0,23900.0,5.0,5.0,0,0,0,0,0,0,1,0
112468,114.93,11.0,15.0,1,500.0,500.0,1200.0,6700.0,5.0,5.0,0,0,0,0,0,0,1,0
112469,94.93,11.0,15.0,1,500.0,500.0,1200.0,6700.0,5.0,5.0,0,0,0,0,0,0,1,0


In [5]:
X_classification = data_classification.drop(["bomb_planted"], axis=1).values
y_classification = data_classification["bomb_planted"].values

In [6]:
from sklearn.preprocessing import MinMaxScaler


minMaxScaler = MinMaxScaler()
minMaxScaler.fit(X_classification)
X_classification = pd.DataFrame(minMaxScaler.transform(X_classification), columns=data_classification.drop(["bomb_planted"], axis=1).columns).values

In [7]:
underSampler = RandomUnderSampler(random_state=42)
X_classification, y_classification = underSampler.fit_resample(X_classification,y_classification)
X_classification= X_classification.reshape(X_classification.shape[0], 1, X_classification.shape[1])

In [8]:
X_classification_train, X_classification_test, y_classification_train, y_classification_test = train_test_split(X_classification,
                                                                                                                y_classification,
                                                                                                                stratify=y_classification,
                                                                                                                test_size=0.2)

In [10]:
model_classification = Network()
model_classification.add(FCLayer(17, 3))
model_classification.add(ActivationLayer(Relu.callFunction, Relu.callDerivative))
model_classification.add(FCLayer(3, 1))
model_classification.add(ActivationLayer(Sigmoid.callFunction, Sigmoid.callDerivative))
model_classification.compile(mse, mse_derivative)
model_classification.fit(X_classification_train, y_classification_train, epochs=50, learning_rate=0.1)
pred_classification = model_classification.predict(X_classification_test)

In [13]:
pred_classification = np.around(pred_classification)
print(classification_report(y_classification_test, pred_classification))
print(confusion_matrix(y_classification_test, pred_classification))

              precision    recall  f1-score   support

           0       0.91      0.98      0.95      2642
           1       0.98      0.91      0.94      2641

    accuracy                           0.95      5283
   macro avg       0.95      0.95      0.95      5283
weighted avg       0.95      0.95      0.95      5283

[[2597   45]
 [ 242 2399]]


In [20]:
data_regression = pd.read_csv("../../data/trip_duration_task_m.csv")
data_regression.drop(["Unnamed: 0"], axis=1, inplace=True)
data_regression.shape

(199494, 5)

In [31]:
y_regression = data_regression['trip_duration'].values
X_regression = data_regression.drop(['trip_duration'], axis=1).values.reshape(data_regression.shape[0], 1, data_regression.shape[1]- 1)

In [32]:

X_regression_train, X_regression_test, y_regression_train, y_regression_test = train_test_split(X_regression,
                                                                                                y_regression,
                                                                                                test_size=0.2)

In [41]:
model_regression = Network()
model_regression.add(FCLayer(4, 32))
model_regression.add(ActivationLayer(Relu.callFunction, Relu.callDerivative))
model_regression.add(FCLayer(3, 1))
model_regression.compile(mse, mse_derivative)
model_regression.fit(X_regression_train, y_regression_train, epochs=50, learning_rate=0.1)


In [44]:
print(mean_absolute_error(y_regression_test, model_regression.predict(X_regression_test)))
print(mean_squared_error(y_regression_test, model_regression.predict(X_regression_test)))

600.5977700830507
103137012.80789636
